In [1]:
import os, websocket, json, csv, dateutil.parser
import pandas as pd 
from sqlalchemy import create_engine, Column, Integer, String, Float
from pathlib import Path
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

from dotenv import load_dotenv

In [74]:
URL = os.getenv("POSTGRES_URL")
USER = os.getenv("POSTGRES_USER")
PWD = os.getenv("POSTGRES_PWD")
DB = os.getenv("POSTGRES_DB")
SSLMODE = "require"
table_init = Path("./schema.sql")

engine = create_engine(f"postgresql://{USER}:{PWD}@{URL}:5432/{DB}")
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [76]:
Base = declarative_base()

class Transaction(Base):
    __tablename__='transactions'

    id = Column(Integer, primary_key=True)
    type = Column(String)
    sequence = Column(Integer)
    product_id = Column(String)
    price = Column(String)
    open_24h = Column(String)
    volume_24h = Column(String)
    low_24h = Column(String)
    high_24h = Column(String)
    volume_30d = Column(String)
    best_bid = Column(String)
    best_ask = Column(String)
    side = Column(String)
    time = Column(String)
    trade_id = Column(Integer)
    last_size = Column(String)

    def __repr__(self):
        return "<Transaction(type='%s', sequence='%d', product_id='%s', price='%s', open_24h='%s', volume_24h='%s', low_24h='%s', high_24h='%s', volume_30d='%s', best_bid='%s', best_ask='%s', side='%s', time='%s', trade_id='%d', last_size='%s')>" % (
            self.type, self.sequence, self.product_id, self.price, self.open_24h, self.volume_24h, self.low_24h, self.high_24h, self.volume_30d, self.best_bid, self.best_ask, self.side, self.time, self.trade_id, self.last_size
        )

In [77]:
to_add = []
def write_db():
    session.add_all(to_add)
    session.commit()
    print("committed")
    to_add = []

In [69]:
def on_open(ws):
    print("opened connection")

    subscribe_message = {
        "type": "subscribe",
        "channels": [
            {
                "name": "ticker",
                "product_ids": [
                    "BTC-USD",
                    "XRP-USD",
                    "ETH-USD"
                ]
            }
        ]
    }

    ws.send(json.dumps(subscribe_message))

def on_message(ws, message):
    tick = json.loads(message)
    to_add.append(Transaction(type=tick['type'], sequence=tick['sequence'], product_id=tick['product_id'], price=tick['price'], open_24h=tick['open_24h'], volume_24h=tick['volume_24h'], low_24h=tick['low_24h'], high_24h=tick['high_24h'], volume_30d=tick['volume_30d'], best_bid=tick['best_bid'], best_ask=tick['best_ask'], side=tick['side'], time=tick['time'], trade_id=tick['trade_id'], last_size=tick['last_size']))
    if len(to_add) >= 10:
        write_db()
    print(len(to_add))

socket = "wss://ws-feed.pro.coinbase.com"
ws = websocket.WebSocketApp(socket, on_open=on_open, on_message=on_message)
ws.run_forever()

opened connection
1
2
3
4
5
6
7
8
9


In [78]:
trn = Transaction(type='ticker',sequence=17174103711,product_id='BTC-USD',price='13848.07', open_24h='13540.11', volume_24h='13165.98633217', low_24h='13424.73', high_24h='14098.92', volume_30d='321801.84181416', best_bid='13848.06', best_ask='13848.07', side='buy', time='2020-10-31T16:37:29.834144Z', trade_id=107361013, last_size='0.01105529')
session.rollback()
session.add(trn)
session.commit()

DataError: (psycopg2.errors.NumericValueOutOfRange) integer out of range

[SQL: INSERT INTO transactions (type, sequence, product_id, price, open_24h, volume_24h, low_24h, high_24h, volume_30d, best_bid, best_ask, side, time, trade_id, last_size) VALUES (%(type)s, %(sequence)s, %(product_id)s, %(price)s, %(open_24h)s, %(volume_24h)s, %(low_24h)s, %(high_24h)s, %(volume_30d)s, %(best_bid)s, %(best_ask)s, %(side)s, %(time)s, %(trade_id)s, %(last_size)s) RETURNING transactions.id]
[parameters: {'type': 'ticker', 'sequence': 17174103711, 'product_id': 'BTC-USD', 'price': '13848.07', 'open_24h': '13540.11', 'volume_24h': '13165.98633217', 'low_24h': '13424.73', 'high_24h': '14098.92', 'volume_30d': '321801.84181416', 'best_bid': '13848.06', 'best_ask': '13848.07', 'side': 'buy', 'time': '2020-10-31T16:37:29.834144Z', 'trade_id': 107361013, 'last_size': '0.01105529'}]
(Background on this error at: http://sqlalche.me/e/13/9h9h)